## Reflexion posterior al Hackaton Caguas

El hackaton fue una tremenda experiencia, donde ademas de aprender, tuvimos el espacio de pensar, discutir y construir soluciones en grupo en favor del sistema de energia electrica de Puerto Rico.

Abundar sobre quienes somos, de donde venimos y que hicimos.

# Índices de confiabilidad eléctrica en Puerto Rico

### Fuente de informacion

Abundar sobre:
- cuales son los indices con los que trabajamos
- quien los produce
- como se producen
- donde se publican
- como se publican
- como entendemos que deberian publicarse y porque

Borrador

Los indices de confiabilidad electrica en Puerto Rico (SAIDI, SAIFI, y CAIFI) son producidos por ??????? (Negociado de Energia de Puerto Rico). No obstante, el formato en que se publican estos indices dificulta el uso de los mismos. 

El formato en que publican estos datos en la actualidad es un archivo de excel con una sola hoja de calculo. En dicha hoja se presentan los indices por periodos.

La hoja (spreadsheet) no mantiene el mismo formato en toda la pagina y contiene errores en las fechas. 

Todo esto que mencionamos tiene unas repercusiones significativas a la hora de analizar, automatizar y tomar decisiones; y se traduce a costos (tiempo y trabajo).

#### Solucion 1: Reformatear la data existente

Nuestro primer objetivo fue reformatear la data existente de tal forma que pueda utilizarse. A continuacion encontraras el proceso para abrir, leer, procesar y limpiar la data publicada. Para mantener la integridad de los datos, descargamos la publicacion en `csv`. *(requerimiento)*

- `read_file`: funcion que abre, lee archivo `csv` y lo transforma a diccionario *(no limpia los datos)*
- `group_data_by_period`: funcion que agrupa la data en periodos *(no corrige errores en las fechas)*
- `group_data_by_region`: funcion que agrupa la data en regiones
- `clean_data`: funcion que produce diccionario con los datos agrupados por periodos y regiones *(no corrige errores en las fechas)*

In [1]:
import csv, re, json

In [2]:
def read_file(filepath):
    with open(filepath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        csv_reader = list(csv_reader)
        period_table = {}

        # starting and endpoint for tables locations
        sp, ep = 3, 10

        # Loop for col
        for j in range(3, 78):
            # define storage
            period_month = ''
            temp_list=[]

            # loop for rows
            for i in range(21):
                if i == 3:
                    temp_table = csv_reader[i][sp:ep]
                    if len(temp_table) > 0:
                        temp_month = temp_table[0].rstrip()
                        date_regex = r"\d{2}-\d{2}-\d{4}"
                        matches = re.finditer(date_regex, temp_month, re.MULTILINE)
                        for matchNum, match in enumerate(matches, start=1):
                            period_month = match.group()
                        temp_list.append(temp_table)
                elif i > 3:
                    temp_table = csv_reader[i][sp:ep]
                    if len(temp_table) > 0:
                        temp_list.append(temp_table)
            period_table[period_month]=temp_list
            # break
            ep+=8
            sp+=8

        temp_data = {}
        for period, table in period_table.items():
            if table and period:
                distribution_header = table[2][:3]
                transmission_header = table[2][4:]

                # Extracting data for Distribution and Transmission
                distribution_data = [[period]]
                transmission_data = [[period]]
                distribution_data.append(distribution_header)
                transmission_data.append(transmission_header)

                for row in table[3:]:
                    distribution_data.append(row[:3])
                    transmission_data.append(row[4:])

                temp_data[period] = {
                    'transmision': transmission_data,
                    'distribucion': distribution_data
                    }
        return temp_data

In [3]:
def group_data_by_period(data):
    grouped_data = {}
    for k, v in data.items():
        data_distribution = v['distribucion']
        data_transmission = v['transmision']
        data_d = {
            row[0].lower().replace(' ', '_'): {
                "saidi": row[1],
                "saifi": row[2]
            }
            for row in data_distribution[2:]
        }
        data_t = {
            row[0].lower().replace(' ', '_'): {
                "saidi": row[1],
                "saifi": row[2]
            }
            for row in data_transmission[2:]
        }
        grouped_data[k] = {
            'transmision': data_t,
            'distribucion': data_d
            }
    return dict(grouped_data)

In [4]:
def group_data_by_region(data):
    grouped_data = {}
    for date, categories in data.items():
        for category, regions in categories.items():
            for region, metrics in regions.items():
                if region not in grouped_data:
                    grouped_data[region] = {}
                if date not in grouped_data[region]:
                    grouped_data[region][date] = {}
                grouped_data[region][date][category] = metrics
    return dict(grouped_data)

In [10]:
def clean_data(temp_data):
    clean_data = {}
    grouped_by_period = group_data_by_period(temp_data)
    grouped_by_region = group_data_by_region(grouped_by_period)
    clean_data['by_period'] = grouped_by_period
    clean_data['by_region'] = grouped_by_region
    return clean_data

##### Ejemplo: Reformatear la data existente

In [6]:
last_report = 'SAIDI_SAIFI_raw_FY24_Q4_v2.csv'
temp_data = read_file(last_report)
clean_data = clean_data(temp_data)
clean_data

{'by_period': {'06-30-2021': {'transmision': {'arecibo': {'saidi': '35.69',
     'saifi': '0.11'},
    'utuado': {'saidi': '80.81', 'saifi': '0.96'},
    'bayamón': {'saidi': '20.86', 'saifi': '0.29'},
    'vega_baja': {'saidi': '22.01', 'saifi': '0.23'},
    'barranquitas': {'saidi': '0.72', 'saifi': '0.02'},
    'caguas': {'saidi': '49.47', 'saifi': '0.74'},
    'humacao': {'saidi': '92.80', 'saifi': '0.23'},
    'canóvanas': {'saidi': '0.00', 'saifi': '0.00'},
    'fajardo': {'saidi': '0.00', 'saifi': '0.00'},
    'aguadilla': {'saidi': '48.04', 'saifi': '0.23'},
    'mayagüez': {'saidi': '175.09', 'saifi': '0.95'},
    'guayama': {'saidi': '0.00', 'saifi': '0.00'},
    'ponce': {'saidi': '169.75', 'saifi': '0.49'},
    'yauco': {'saidi': '148.44', 'saifi': '1.36'},
    'san_juan': {'saidi': '31.37', 'saifi': '0.16'}},
   'distribucion': {'arecibo': {'saidi': '168.59', 'saifi': '0.45'},
    'utuado': {'saidi': '661.64', 'saifi': '0.72'},
    'bayamón': {'saidi': '165.86', 'saifi': '

#### Solucion 2: Mostrar datos reformateados

Nuestro segundo objetivo fue desarrollar un API para mostrar los datos reformateados. Para esto creamos una aplicacion en Flask que carga los datos y los expone en tres rutas:

- `/` - ruta de bienvenida
- `/indices/todos` - ruta muestra los indices agrupados por periodos y regiones
- `/indices/todos/por_periodos` - ruta muestra los indices agrupados por periodos
- `/indices/todos/por_regiones` - ruta muestra los indices agrupados por regiones

El codigo a continuacion es un ejemplo de la aplicacion en flask:

- ```
    from flask import Flask, jsonify
    import json

    app = Flask(__name__)

    last_report = 'SAIDI_SAIFI_raw_FY24_Q4_v2.csv'
    temp_data = read_file(last_report)
    clean_data = clean_data(temp_data)

    @app.route('/')
    def hello_world():
        welcome = 'Este API proporciona acceso a datos sobre los índices de confiabilidad eléctrica en Puerto Rico (SAIDI, SAIFI, y CAIFI). Estos índices permiten evaluar la duración y frecuencia de interrupciones del servicio eléctrico.'
        response = app.response_class(
            response=json.dumps({"message": welcome}, ensure_ascii=False).encode('utf8'),
            mimetype='application/json; charset=utf-8'
        )
        return response

    # ------------------------ #
    # NUEVO...
    # En español, puñeee...
    # ------------------------ #

    @app.route('/indices/todos')
    def todos_indices():
        data = clean_data
        # Requerido para incluir valores en español
        response = jsonify(data)
        response.headers['Content-Type'] = 'application/json; charset=utf-8'
        return response

    @app.route('/indices/todos/por_periodos')
    def indices_periodos():
        data = clean_data['by_period']    
        # Requerido para incluir valores en español
        response = jsonify(data)
        response.headers['Content-Type'] = 'application/json; charset=utf-8'
        return response

    @app.route('/indices/todos/por_regiones')
    def indices_regiones():
        data = clean_data['by_region']    
        # Requerido para incluir valores en español
        response = jsonify(data)
        response.headers['Content-Type'] = 'application/json; charset=utf-8'
        return response

    if __name__ == '__main__':
        app.run(debug=True, threaded=True)
    ```

### Notas finales

Nos preguntamos entonces, para que se publican los indices? Sera para utilizarlos en la toma de decisiones o meramente para cumplir con las leyes (aka requirimiento para cobrar).

Si es para la toma de decisiones, deberian estar en un formato que permita su uso.

Nuestro primer objetivo fue reformatear la data existente de tal forma que pueda utilizarse.